# Trabalho 05 - Classificação

In [151]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
from sklearn.impute import KNNImputer

In [152]:
from sklearn.calibration import LabelEncoder


def load_and_preprocess_data(inputting_method):
  df = pd.read_csv('data/trabalho5_dados_4.csv')
  
  # display(((df == 'MD') | df.isna()).sum())
  df = df.replace('MD', np.nan)
  df = fill_missing_data(df, method=inputting_method)
  df['Quantitative Ability 1'] = df['Quantitative Ability 1'].astype(float)
  df['Analytical Skills 1'] = df['Analytical Skills 1'].astype(float)
  df['Domain Skills 1'] = df['Domain Skills 1'].astype(float)
  df['Year of Birth'] = df['Year of Birth'].map(lambda x: x[1:]).astype(int)
  df['10th Completion Year'] = df['10th Completion Year'].map(lambda x: x[1:]).astype(int)
  df['12th Completion year'] = df['12th Completion year'].map(lambda x: x[1:]).astype(int)
  df['Year of Completion of college'] = df[' Year of Completion of college'].map(lambda x: x[1:]).astype(int)
  df['10th percentage'] = df[' 10th percentage']
  df['12th percentage'] = df[' 12th percentage']
  df['College percentage'] = df[' College percentage']
  df['English 1'] = df[' English 1']

  df.drop(' Year of Completion of college', axis=1, inplace=True)
  df.drop(' 10th percentage', axis=1, inplace=True)
  df.drop(' 12th percentage', axis=1, inplace=True)
  df.drop(' College percentage', axis=1, inplace=True)
  df.drop(' English 1', axis=1, inplace=True)


  # df = encode_string_columns(df)
  # df = df.drop_duplicates(df, keep='first')

  return df


def fill_missing_data(df, method='knn', n_neighbors=5):
  if method == 'simple':
    for col in df.columns:
      if df[col].dtype in ['float64', 'int64']:
        df[col] = df[col].fillna(df[col].mean())
      else:
        if df[col].notna().any():
          df[col] = df[col].fillna(df[col].mode()[0])

  elif method == 'knn':
    # Separate numeric and non-numeric columns
    df_numeric = df.select_dtypes(include=['float64', 'int64'])
    df_non_numeric = df.select_dtypes(exclude=['float64', 'int64'])

    # Apply get_dummies for non-numeric columns to perform one-hot encoding
    df_non_numeric_dummies = pd.get_dummies(df_non_numeric, drop_first=False)

    # Impute using KNN
    imputer_numeric = KNNImputer(n_neighbors=n_neighbors)
    imputer_non_numeric = KNNImputer(n_neighbors=1)
    
    df_imputed_numeric = pd.DataFrame(
      imputer_numeric.fit_transform(df_numeric),
      columns=df_numeric.columns,
      index=df_numeric.index
    )

    df_imputed_non_numeric = pd.DataFrame(
      imputer_non_numeric.fit_transform(df_non_numeric_dummies),
      columns=df_non_numeric_dummies.columns,
      index=df_non_numeric_dummies.index
    )
    # Reverse the one-hot encoding by getting the most frequent category for each column
    df_non_numeric_imputed = pd.DataFrame(index=df_imputed_non_numeric.index)
    for col in df_non_numeric.columns:
      vals_col = df_non_numeric[col].dropna().unique()
      
      dummies = [col + '_' + str(val) for val in vals_col]
      dummies_values = df_imputed_non_numeric[dummies].idxmax(axis=1).apply(lambda x: x.split('_')[-1])
      df_non_numeric_imputed[col] = dummies_values

    # Combine numeric and non-numeric back to the original DataFrame
    df = pd.concat([df_imputed_numeric, df_non_numeric_imputed], axis=1)

  else:
    raise ValueError("Invalid method. Choose from 'simple' or 'knn'.")

  return df


def encode_string_columns(df, categorical_columns: list[str]):
  for col in categorical_columns:
    # Get the unique values in the column
    unique_values = df[col].dropna().unique()
    
    if col == 'x':
      df['dummy_x'] = df[col].str.lower().map({'nao':0, 'as vezes':1, 'frequentemente':2, 'sempre': 3})
      continue
      
    if len(unique_values) == 2:
      label_encoder = LabelEncoder()
      df[f"dummy_{col}"] = label_encoder.fit_transform(df[col])
      continue
    else:
      df_dummies = pd.get_dummies(df[col], prefix=f"dummy_{col}")
      df = pd.concat([df, df_dummies], axis=1)
      continue
        
  return df

In [153]:
numerical_columns = [
    'Candidate ID',
    'Number of characters in Original Name',
    '10th percentage',
    '12th percentage',
    'College percentage',
    'English 1',
    'English 2',
    'English 3',
    'English 4',
    'Quantitative Ability 1',
    'Quantitative Ability 2',
    'Quantitative Ability 3',
    'Quantitative Ability 4',
    'Domain Skills 1',
    'Domain Skills 2',
    'Domain Test 3',
    'Domain Test 4',
    'Analytical Skills 1',
    'Analytical Skills 2',
    'Analytical Skills 3',
    'Year of Birth',
    '10th Completion Year',
    '12th Completion year',
    'Year of Completion of college'
]

categorical_columns = [
    'Month of Birth',
    'Gender',
    'State (Location)',
    'Degree of study',
    'Specialization in study'
]

y_column = 'Performance'

# Ignored columns: Name

In [154]:
df = load_and_preprocess_data('knn')
df


,Candidate ID,Number of characters in Original Name,English 2,English 3,English 4,Quantitative Ability 2,Quantitative Ability 3,Quantitative Ability 4,Domain Skills 2,Domain Test 3,...,Specialization in study,Quantitative Ability 1,Domain Skills 1,Analytical Skills 1,Performance,Year of Completion of college,10th percentage,12th percentage,College percentage,English 1
0,1056.0,11.0,57.961783,60.000000,59.509202,58.083832,74.301676,76.878613,61.151079,53.103448,...,B,65.573770,48.214286,81.818182,BP,20,74.20,69.60,73.20,37.735849
1,1058.0,12.0,73.248408,69.696970,68.098160,54.491018,59.776536,60.693642,68.345324,58.620690,...,G,37.704918,46.428571,89.090909,BP,20,80.33,72.20,76.32,54.716981
2,1059.0,13.0,89.808917,60.000000,61.963190,52.095808,68.715084,64.161850,64.028777,57.241379,...,B,49.180328,44.642857,36.363636,MP,20,77.80,74.40,67.90,66.037736
3,1062.0,16.0,60.509554,55.151515,69.325153,77.245509,67.597765,56.069364,76.978417,72.413793,...,B,59.016393,64.285714,63.636364,BP,20,77.50,72.40,72.40,41.509434
4,1063.0,11.0,89.808917,78.181818,68.098160,66.467066,64.245810,60.693642,82.733813,58.620690,...,C,52.459016,69.642857,49.090909,BP,20,80.53,75.83,66.31,71.698113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,85.0,16.0,65.605096,58.787879,57.055215,60.479042,64.245810,57.225434,84.172662,48.965517,...,B,44.262295,53.571429,72.727273,BP,20,78.60,88.58,77.00,32.075472
463,474.0,9.0,65.605096,89.090909,58.282209,74.850299,58.659218,61.849711,59.712230,58.620690,...,B,42.622951,48.214286,87.272727,BP,20,89.00,71.00,65.60,83.018868
464,318.0,13.0,79.617834,75.757576,60.736196,68.862275,45.251397,50.289017,79.856115,73.793103,...,B,26.229508,58.928571,49.090909,MP,20,84.40,79.20,78.03,60.377358
465,162.0,19.0,88.535032,63.636364,64.417178,76.047904,81.005587,86.127168,65.467626,48.965517,...,L,77.049180,44.642857,81.818182,BP,20,95.17,94.33,71.20,49.056604


In [155]:
df[numerical_columns]

,Candidate ID,Number of characters in Original Name,10th percentage,12th percentage,College percentage,English 1,English 2,English 3,English 4,Quantitative Ability 1,...,Domain Skills 2,Domain Test 3,Domain Test 4,Analytical Skills 1,Analytical Skills 2,Analytical Skills 3,Year of Birth,10th Completion Year,12th Completion year,Year of Completion of college
0,1056.0,11.0,74.20,69.60,73.20,37.735849,57.961783,60.000000,59.509202,65.573770,...,61.151079,53.103448,89.781022,81.818182,68.354430,88.505747,7,14,16,20
1,1058.0,12.0,80.33,72.20,76.32,54.716981,73.248408,69.696970,68.098160,37.704918,...,68.345324,58.620690,63.503650,89.090909,88.607595,88.505747,7,14,16,20
2,1059.0,13.0,77.80,74.40,67.90,66.037736,89.808917,60.000000,61.963190,49.180328,...,64.028777,57.241379,64.963504,36.363636,70.886076,19.540230,8,14,16,20
3,1062.0,16.0,77.50,72.40,72.40,41.509434,60.509554,55.151515,69.325153,59.016393,...,76.978417,72.413793,75.182482,63.636364,68.354430,63.218391,7,14,16,20
4,1063.0,11.0,80.53,75.83,66.31,71.698113,89.808917,78.181818,68.098160,52.459016,...,82.733813,58.620690,94.160584,49.090909,60.759494,44.827586,8,14,16,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,85.0,16.0,78.60,88.58,77.00,32.075472,65.605096,58.787879,57.055215,44.262295,...,84.172662,48.965517,76.642336,72.727273,63.291139,79.310345,8,14,16,20
463,474.0,9.0,89.00,71.00,65.60,83.018868,65.605096,89.090909,58.282209,42.622951,...,59.712230,58.620690,66.423358,87.272727,81.012658,88.505747,6,13,15,20
464,318.0,13.0,84.40,79.20,78.03,60.377358,79.617834,75.757576,60.736196,26.229508,...,79.856115,73.793103,62.043796,49.090909,75.949367,37.931034,7,14,16,20
465,162.0,19.0,95.17,94.33,71.20,49.056604,88.535032,63.636364,64.417178,77.049180,...,65.467626,48.965517,70.802920,81.818182,88.607595,74.712644,7,14,16,20
